In [16]:
import pandas as pd
porto1_df=pd.read_csv('porto1.csv')

import numpy as np
porto1_df.replace(-1,np.NaN,inplace=True)

porto1_df.isnull().sum()
porto1_df.columns
porto1_df.shape

y=porto1_df['target']
porto1_df.drop(columns='target',inplace=True)
y

0        0
1        0
2        0
3        0
4        0
        ..
51770    0
51771    0
51772    0
51773    0
51774    0
Name: target, Length: 51775, dtype: int64

In [33]:
miss_val_df= pd.DataFrame({'column_name':porto1_df.columns,
                          'missing_value_count':porto1_df.isnull().sum(),
                          'missing_percentage':porto1_df.isnull().sum()/porto1_df.shape[0]*100})
miss_val_df=miss_val_df.sort_values(by='missing_percentage',ascending=False)
drop_columns=list(miss_val_df[miss_val_df['missing_percentage']>50]['column_name'])
porto1_df.drop(columns=drop_columns,inplace=True)
porto1_df.shape

porto1_int_float = [i for i in porto1_df.columns if porto1_df[i].dtype == 'int64' or porto1_df[i].dtype == 'float64']
len(porto1_int_float)
drop_columns

porto1_object = [i for i in porto1_df.columns if porto1_df[i].dtype == object]
porto1_object
len(porto1_object)
porto1_df.shape
length = len(porto1_object) + len(porto1_int_float)
length

for i in porto1_int_float:
    porto1_df[i].fillna(porto1_df[i].mean(),inplace=True)
for i in porto1_int_float:
    porto1_df[i].fillna(porto1_df[i].mode()[0],inplace=True)

len(porto1_df.columns)

porto1_df.isnull().sum()

id               0
ps_ind_01        0
ps_ind_02_cat    0
ps_ind_03        0
ps_ind_04_cat    0
ps_ind_05_cat    0
ps_ind_06_bin    0
ps_ind_07_bin    0
ps_ind_08_bin    0
ps_ind_09_bin    0
ps_ind_10_bin    0
ps_ind_11_bin    0
ps_ind_12_bin    0
ps_ind_13_bin    0
ps_ind_14        0
ps_ind_15        0
ps_ind_16_bin    0
ps_ind_17_bin    0
dtype: int64

In [35]:
porto1_df.shape

(51775, 18)

In [42]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(porto1_df,y,test_size=0.2,random_state=26)
for col in X_train.columns:
        if X_train[col].dtype==object:
            X_train[col]=X_train[col].fillna(X_train[col].mode()[0]).astype(object)
            X_test[col]=X_test[col].fillna(X_train[col].mode()[0]).astype(object)
        else:
            X_train[col]=X_train[col].fillna(X_train[col].mean())
            X_test[col]=X_test[col].fillna(X_train[col].mean())

X_train
Y_train

X_test
Y_test

X_train.shape

C:\Users\my laptop\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\my laptop\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


(41420, 18)

In [49]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

logreg.fit(X_train,Y_train)
y_pred = logreg.predict(X_test)

comp_df = pd.DataFrame({'Actual_values': Y_test,'Predicted':y_pred})
comp_df

y_pred_proba = logreg.predict_proba(X_test)[::,1]
y_pred_proba

from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,roc_auc_score,roc_curve
confusion_matrix(Y_test,y_pred)


array([[9968,    0],
       [ 387,    0]], dtype=int64)

In [51]:
!pip install imblearn

In [53]:
import imblearn
from imblearn.over_sampling import SMOTE 

smote=SMOTE(sampling_strategy=0.40)
Xsmote,Ysmote=smote.fit_resample(X_train,Y_train)


In [54]:
X_train.shape, Xsmote.shape
Ysmote.value_counts(normalize=True)

0    0.714293
1    0.285707
Name: target, dtype: float64